In [40]:
import re

log_entries = []

# Read the log file
with open('data/haproxytmp.log', 'r') as file:
    log_entries = file.readlines()

# Define regular expression for field extraction
regex_pattern = r'^(?P<timestamp>\w+\s+\d+\s\d+:\d+:\d+)\s+(?P<server>\S+)\s+haproxy\[\d+\]:\s(?P<client_ip>[\d.:]+):(?P<client_port>\d+)\s+\[(?P<datetime>[^\]]+)\]\s+(?P<frontend>\S+)~\s+(?P<backend>[^\s/]+)/(?P<backendservername>[^\s]+)\s+(?P<timing>[^\s]+)\s+(?P<http_status>\d+)\s+(?P<bytes_read>\d+)\s+.*\s+(?P<act_conn>\d+)/(?P<fe_conn>\d+)/(?P<be_conn>\d+)/(?P<srv_conn>\d+)/(?P<retries>\d+)\s+.*\s+"(?P<request_line>[^"]+)"'

fields = []

# Iterate through log entries
for log_entry in log_entries:
    # Extract fields using regular expression
    match = re.match(regex_pattern, log_entry)
    if match:
        fields.append({
            'timestamp': match.group('timestamp'),
            'server': match.group('server'),
            'client_ip': match.group('client_ip'),
            'client_port': match.group('client_port'),
            'datetime': match.group('datetime'),
            'frontend': match.group('frontend'),
            'backend': match.group('backend'),
            'backendservername': match.group('backendservername'),
            'timing': match.group('timing'),
            'http_status': match.group('http_status'),
            'bytes_read': match.group('bytes_read'),
            'request_cookie': '-',
            'response_cookie': '-',
            'termination_state': '-',
            'act_conn': match.group('act_conn'),
            'fe_conn': match.group('fe_conn'),
            'be_conn': match.group('be_conn'),
            'srv_conn': match.group('srv_conn'),
            'act_sess': '0',
            'fe_sess': '0',
            'be_sess': '0',
            'srv_sess': '0',
            'retries': match.group('retries'),
            'request_line': match.group('request_line')
        })

# Print extracted fields
for field in fields:
    print(field)


{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '41.210.186.148', 'client_port': '51732', 'datetime': '06/Jun/2023:07:35:12.147', 'frontend': 'www-https', 'backend': 'interswitchstore_backend', 'backendservername': 'interswitchstore_webserver', 'timing': '371/0/1/37/409', 'http_status': '200', 'bytes_read': '217', 'act_conn': '452', 'fe_conn': '452', 'be_conn': '0', 'srv_conn': '1', 'retries': '0', 'request_line': 'POST /interswitchstore/device/remoteConfigs HTTP/1.1'}
{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '196.13.203.201', 'client_port': '59225', 'datetime': '06/Jun/2023:07:35:12.378', 'frontend': 'www-https', 'backend': 'api_backend', 'backendservername': 'api_webserver2', 'timing': '3/0/1/187/191', 'http_status': '200', 'bytes_read': '314', 'act_conn': '452', 'fe_conn': '452', 'be_conn': '0', 'srv_conn': '1', 'retries': '0', 'request_line': 'POST /api/v1A/svapayments/validateCustomer HTTP/1.1'}
{